In [3]:
import numpy as np 
import tensorflow as tf 
import tensorflow_recommenders as tfrs 
import tensorflow_datasets as tfds 

ERROR:absl:module 'tensorflow_datasets.core' has no attribute 'utils'
Traceback (most recent call last):
  File "c:\Users\bpadmin\anaconda3\envs\tensorflow_cuda\lib\site-packages\tensorflow_datasets\__init__.py", line 59, in <module>
    from tensorflow_datasets import audio
  File "c:\Users\bpadmin\anaconda3\envs\tensorflow_cuda\lib\site-packages\tensorflow_datasets\audio\__init__.py", line 19, in <module>
    from tensorflow_datasets.audio.commonvoice import CommonVoice
  File "c:\Users\bpadmin\anaconda3\envs\tensorflow_cuda\lib\site-packages\tensorflow_datasets\audio\commonvoice.py", line 27, in <module>
    import tensorflow_datasets.public_api as tfds
  File "c:\Users\bpadmin\anaconda3\envs\tensorflow_cuda\lib\site-packages\tensorflow_datasets\public_api.py", line 57, in <module>
    deprecated = core.utils.docs.deprecated(deprecated)
AttributeError: module 'tensorflow_datasets.core' has no attribute 'utils'


In [2]:
%%capture

ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

NameError: name 'tfds' is not defined

In [16]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "timestamp": x["timestamp"],
})

movies = movies.map(lambda x: x["movie_title"])

In [17]:
timestamps = np.concatenate(list(ratings.map(lambda x: x["timestamp"]).batch(100)))

max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

unique_movie_titles = np.unique(np.concatenate(list(movies.batch(1000))))
unique_user_ids = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["user_id"]))))

steps:
1. preprocessing
2. define query and candidate model

In [6]:
next(movies.take(1).as_numpy_iterator())

{'movie_genres': array([4], dtype=int64),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}

In [14]:
next(ratings.take(1).as_numpy_iterator())
# next((ratings.map(lambda x: x['movie_id'] == '1681')).take(1).as_numpy_iterator())

{'bucketized_user_age': 45.0,
 'movie_genres': array([7], dtype=int64),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}

# UserModel

In [18]:
class UserModel(tf.keras.Model):

  def __init__(self, use_timestamps):
    super().__init__()

    self._use_timestamps = use_timestamps

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])

    if use_timestamps:
      self.timestamp_embedding = tf.keras.Sequential([
          tf.keras.layers.Discretization(timestamp_buckets.tolist()),
          tf.keras.layers.Embedding(len(timestamp_buckets) + 1, 32),
      ])
      self.normalized_timestamp = tf.keras.layers.Normalization(
          axis=None
      )

      self.normalized_timestamp.adapt(timestamps)


  def call(self, inputs):
    if not self._use_timestamps:
      return self.user_embedding(inputs["user_id"])

    return tf.concat([
        self.user_embedding(inputs["user_id"]),
        self.timestamp_embedding(inputs["timestamp"]),
        tf.reshape(self.normalized_timestamp(inputs["timestamp"]), (-1, 1)), 
    ], axis=1)

# ItemModel

In [19]:
class MovieModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, 32)
    ])

    self.title_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])
    
    self.title_vectorizer.adapt(movies)

  def call(self, titles):
    return tf.concat([
        self.title_embedding(titles),
        self.title_text_embedding(titles),
    ], axis=1)

# recommender model

In [26]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, use_timestamps):
    super().__init__()
    self.query_model = tf.keras.Sequential([
      UserModel(use_timestamps),
      tf.keras.layers.Dense(32)
    ])
    self.candidate_model = tf.keras.Sequential([
      MovieModel(),
      tf.keras.layers.Dense(32)
    ])
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.candidate_model),
            # ks = [10]
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"],
        "timestamp": features["timestamp"],
    })
    movie_embeddings = self.candidate_model(features["movie_title"])

    return self.task(query_embeddings, movie_embeddings)

# data split

In [21]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

cached_train = train.shuffle(100_000).batch(2048)
cached_test = test.batch(4096).cache()

In [39]:
import pprint
pprint.pprint(next(train.take(1).as_numpy_iterator()))

{'movie_title': b'Postman, The (1997)',
 'timestamp': 885409515,
 'user_id': b'681'}


In [27]:
model = MovielensModel(use_timestamps=False)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

Epoch 1/3
40/40 [==============================] - 6s 117ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0150 - factorized_top_k/top_5_categorical_accuracy: 0.0272 - factorized_top_k/top_10_categorical_accuracy: 0.0371 - factorized_top_k/top_50_categorical_accuracy: 0.0955 - factorized_top_k/top_100_categorical_accuracy: 0.1596 - loss: 14570.5523 - regularization_loss: 0.0000e+00 - total_loss: 14570.5523
Epoch 2/3
40/40 [==============================] - 5s 121ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0033 - factorized_top_k/top_5_categorical_accuracy: 0.0153 - factorized_top_k/top_10_categorical_accuracy: 0.0276 - factorized_top_k/top_50_categorical_accuracy: 0.1232 - factorized_top_k/top_100_categorical_accuracy: 0.2278 - loss: 14120.1746 - regularization_loss: 0.0000e+00 - total_loss: 14120.1746
Epoch 3/3
5/5 [==============================] - 1s 172ms/step - factorized_top_k/top_1_categorical_accuracy: 4.0000e-04 - factorized_top_k/top_5_categorical_accuracy

In [41]:
model = MovielensModel(use_timestamps=True)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

Epoch 1/3
40/40 [==============================] - 6s 115ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0110 - factorized_top_k/top_5_categorical_accuracy: 0.0231 - factorized_top_k/top_10_categorical_accuracy: 0.0333 - factorized_top_k/top_50_categorical_accuracy: 0.0956 - factorized_top_k/top_100_categorical_accuracy: 0.1626 - loss: 14533.5506 - regularization_loss: 0.0000e+00 - total_loss: 14533.5506
Epoch 2/3
40/40 [==============================] - 5s 116ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0034 - factorized_top_k/top_5_categorical_accuracy: 0.0175 - factorized_top_k/top_10_categorical_accuracy: 0.0341 - factorized_top_k/top_50_categorical_accuracy: 0.1442 - factorized_top_k/top_100_categorical_accuracy: 0.2621 - loss: 13937.2593 - regularization_loss: 0.0000e+00 - total_loss: 13937.2593
Epoch 3/3
5/5 [==============================] - 1s 170ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0011 - factorized_top_k/top_5_categorical_accuracy: 0.

In [29]:
raw_acc = model.evaluate(cached_train, return_dict=True)

40/40 [==============================] - 5s 114ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0028 - factorized_top_k/top_5_categorical_accuracy: 0.0239 - factorized_top_k/top_10_categorical_accuracy: 0.0472 - factorized_top_k/top_50_categorical_accuracy: 0.1896 - factorized_top_k/top_100_categorical_accuracy: 0.3160 - loss: 13678.8459 - regularization_loss: 0.0000e+00 - total_loss: 13678.8459


K categorical accuracy: how often the true candidate is in the top K candidates for a given query.

In [40]:
pprint.pprint(raw_acc)

{'factorized_top_k/top_100_categorical_accuracy': 0.31603750586509705,
 'factorized_top_k/top_10_categorical_accuracy': 0.047187499701976776,
 'factorized_top_k/top_1_categorical_accuracy': 0.0027874999213963747,
 'factorized_top_k/top_50_categorical_accuracy': 0.18963749706745148,
 'factorized_top_k/top_5_categorical_accuracy': 0.023887500166893005,
 'loss': 557.4373168945312,
 'regularization_loss': 0,
 'total_loss': 557.4373168945312}
